In [1]:
import pandas as pd
import numpy as np

# https://www.kaggle.com/uciml/student-alcohol-consumption 

#### Celem naszej analizy jest ustalenie jak często uczniowie spożywają alkohol w dni szkolne - $Dalc$ oraz w weekendy - $Walc$. Na stronie ze zbiorem danych znajduje się dokładny opis parametrów. Jako, że jest to spotkanie związane z kursem Machine Learning by Stanford University, użyte metody uczenia maszynowego nie mają na celu stworzyć najlepszego modelu, lecz zaprezentować praktyczną aplikację poznanych narzędzi - w tym przypadku regresji liniowej.

# Wczytanie danych z student-mat.csv i student-por.csv

In [2]:
mat = pd.read_csv("student-mat.csv")
por = pd.read_csv("student-por.csv")
table = mat.append(por)
table = table.sample(table.shape[0])
table = table.drop(columns=["school","Mjob","Fjob","reason","guardian"])
table.shape

(1044, 28)

# Faktoryzacja danych w postaci tekstu

Sprowadzamy stringi do wartości binarnych.

In [3]:
def stringToInt(df,name):
    df.loc[:,name] = pd.factorize(df.loc[:,name])[0]

namesToDrop = ["sex","address","famsize","Pstatus","schoolsup","famsup",
               "paid","activities","nursery","higher","romantic","internet"]

for name in namesToDrop:
    stringToInt(table, name)

# Podział na $y_{Walc}$, $y_{Dalc}$ i $X$ 

In [4]:
Dalc = np.array(table.loc[:,"Dalc"])
Walc = np.array(table.loc[:,"Walc"])
X = table.drop(columns = ["Dalc","Walc"]).dropna()
X = np.c_[np.ones(X.shape[0]), np.array(X)]

In [5]:
Xtrain = X[0:350,:]
Walctrain = Walc[0:350]
Dalctrain = Dalc[0:350]

In [6]:
Xtest = X[350:395,:]
Walctest = Walc[350:395]
Dalctest = Dalc[350:395]

# Spożywanie alkoholu w weekendy
# $$ \Theta_{Walc} = (X^TX)^{-1}X^Ty$$

In [7]:
ThetaWalc = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Xtrain.transpose(),Xtrain)),Xtrain.transpose()),Walctrain)
result = np.around(np.matmul(Xtest,ThetaWalc))

# Dokładna sprawność, porównanie jeden do jednego

In [8]:
print("Model osiągnął skuteczność: {0:.2f}".format(sum(result == Walctest)/Walctest.size))

Model osiągnął skuteczność: 0.36


# Przybliżona sprawność, tzn. wynik aktualna wartość $\pm1$ 

In [9]:
print("Przy klasyfikacji rozmytej, model osiągnął skuteczność: {0:.2f}".
      format(sum(abs(result-Walctest) <= 1)/Walctest.size))

Przy klasyfikacji rozmytej, model osiągnął skuteczność: 0.80


# Spożywanie alkoholu w dni powszednie
# $$ \Theta_{Dalc} = (X^TX)^{-1}X^Ty$$

In [10]:
ThetaDalc = np.matmul(np.matmul(np.linalg.pinv(np.matmul(Xtrain.transpose(),Xtrain)),Xtrain.transpose()),Dalctrain)
result = np.around(np.matmul(Xtest,ThetaDalc))

# Dokładna sprawność, porównanie jeden do jednego

In [11]:
print("Model osiągnął skuteczność: {0:.2f}".format(sum(result == Dalctest)/Dalctest.size))

Model osiągnął skuteczność: 0.47


# Przybliżona sprawność, tzn. wynik aktualna wartość $\pm1$ 

In [12]:
print("Przy klasyfikacji rozmytej, model osiągnął skuteczność: {0:.2f}".
      format(sum(abs(result-Dalctest) <= 1)/Dalctest.size))

Przy klasyfikacji rozmytej, model osiągnął skuteczność: 0.91
